In [1]:
from dadude.config import default_storage_config

In [2]:
test_patsnap_config = default_storage_config.copy()
test_patsnap_config["AWS_ACCESS_KEY_ID"] = "iamdev"
test_patsnap_config["AWS_SECRET_ACCESS_KEY"] = "xzd19950506"
test_patsnap_config["AWS_ENDPOINT_URL"] = "http://192.168.18.206:9000"

In [3]:
from deltalake import DeltaTable
import polars as pl

In [6]:
df = pl.DataFrame({"foo": [1, 2], "bar": ["1", "2"]})
for i in range(2):
    df.write_delta(
        "../db/test_cdf",
        mode="append",
        delta_write_options={
            "configuration": {"delta.enableChangeDataFeed": "true"},
            "engine": "rust",
        },
    )

dt = DeltaTable("../db/test_cdf")
pl.from_arrow(dt.load_cdf())

foo,bar,_change_type,_commit_version,_commit_timestamp
i64,str,str,i64,datetime[ms]
1,"""1""","""insert""",1,2024-11-20 05:47:49.052
2,"""2""","""insert""",1,2024-11-20 05:47:49.052
1,"""1""","""insert""",0,2024-11-20 05:47:49.049
2,"""2""","""insert""",0,2024-11-20 05:47:49.049
